In [1]:
%cd /opt

/opt


In [2]:
!echo Untarring Kaldi

Untarring Kaldi


In [3]:
%%capture
!tar xvf /kaggle/input/extract-prebuilt-kaldi-from-docker/kaldi.tar

In [4]:
%cd kaldi/egs

/opt/kaldi/egs


In [5]:
!git clone https://github.com/danijel3/ClarinStudioKaldi

Cloning into 'ClarinStudioKaldi'...
remote: Enumerating objects: 778, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 778 (delta 0), reused 0 (delta 0), pack-reused 775
Receiving objects: 100% (778/778), 35.26 MiB | 17.69 MiB/s, done.
Resolving deltas: 100% (262/262), done.


In [6]:
%cd ClarinStudioKaldi

/opt/kaldi/egs/ClarinStudioKaldi


In [7]:
!echo getting cudatoolkit 10.0

getting cudatoolkit 10.0


In [8]:
!timeout 10m conda install --override-channels -c anaconda cudatoolkit=10.0.130 -y

Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | 

In [9]:
import os
os.environ['LD_LIBRARY_PATH'] = '/opt/conda/lib:/opt/kaldi/tools/openfst-1.6.7/lib:/opt/kaldi/src/lib'

In [10]:
!cat path.sh|sed -e 's/~\/apps/\/opt/' > tmp
!mv tmp path.sh

In [11]:
!echo > local_clarin/clarin_pl_clean.sh

In [12]:
!ln -s ../wsj/s5/steps
!ln -s ../wsj/s5/conf
!ln -s ../wsj/s5/local
!ln -s ../wsj/s5/utils

In [13]:
!echo "copying data and exp from last run" 

copying data and exp from last run


In [14]:
!cp -r /kaggle/input/kaldi-clarinstudio-polish-ivectors/data /kaggle/working/
!cp -r /kaggle/input/kaldi-clarinstudio-polish-ivectors/exp /kaggle/working/

In [15]:
!ln -s /kaggle/working/exp
!ln -s /kaggle/working/data

In [16]:
!cp exp/tri3b/35.mdl exp/tri3b/final.mdl
!cp exp/tri3b/35.occs exp/tri3b/final.occs

In [17]:
!echo Setup done

Setup done


In [18]:
%%writefile run.sh
. path.sh

export nj=40 ##number of concurrent processes
export nj_test=30 ## number of concurrent processes for test has to be <=30

#Performing tests on all the above systems:

#./local/nnet3/run_ivector_common.sh --nj $nj_test --train_set train --test_sets test --gmm tri3b

./local_clarin/clarin_tdnn.sh
steps/nnet3/decode.sh --nj $nj_test --num-threads 4 --online-ivector-dir exp/nnet3/ivectors_test_hires \
          exp/tri3b/graph data/test_hires exp/nnet3/tdnn1a_sp/decode
./steps/oracle_wer.sh data/test_hires data/lang exp/nnet3/tdnn1a_sp/decode
./steps/lmrescore_const_arpa.sh data/lang_test data/lang_carpa data/test_hires exp/nnet3/tdnn1a_sp/decode exp/nnet3/tdnn1a_sp/decode_rs

# Same as above but using the chain framework - trains about the same, works much faster as has lower WER
./local_clarin/clarin_chain_tdnn.sh
./steps/oracle_wer.sh data/test_hires data/lang exp/chain/tdnn1f_sp/decode
./steps/lmrescore_const_arpa.sh data/lang_test data/lang_carpa data/test_hires exp/chain/tdnn1f_sp/decode exp/chain/tdnn1f_sp/decode_rs

# Getting results
find exp -name best_wer | while read f ; do cat $f ; done | sort -k2nr
find exp -name oracle_wer | while read f ; do echo -n "$f: " ; cat $f ; done | sort -k2nr

Overwriting run.sh


In [19]:
!bash run.sh

./local_clarin/clarin_tdnn.sh: creating neural net configs using the xconfig parser
tree-info exp/tri3b/tree 
steps/nnet3/xconfig_to_configs.py --xconfig-file exp/nnet3/tdnn1a_sp/configs/network.xconfig --config-dir exp/nnet3/tdnn1a_sp/configs/
nnet3-init: error while loading shared libraries: libcudart.so.10.0: cannot open shared object file: No such file or directory
Traceback (most recent call last):
  File "steps/nnet3/xconfig_to_configs.py", line 333, in <module>
    main()
  File "steps/nnet3/xconfig_to_configs.py", line 327, in main
    existing_model=args.existing_model)
  File "steps/nnet3/xconfig_to_configs.py", line 278, in check_model_contexts
    config_dir, file_name))
  File "steps/libs/common.py", line 158, in execute_command
    p.returncode, command))
Exception: Command exited with status 127: nnet3-init  exp/nnet3/tdnn1a_sp/configs//init.config exp/nnet3/tdnn1a_sp/configs//init.raw
steps/nnet3/decode.sh --nj 30 --num-threads 4 --online-ivector-dir exp/nnet3/ivectors_